#Image Classification with Convolution Neural Network

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5),(0.5))])

trainset = torchvision.datasets.MNIST(root = './data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3)
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.conv3 = nn.Conv2d(64, 64, 3)
    self.conv4 = nn.Conv2d(64, 64, 3)
    self.fc1 = nn.Linear(4096, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, (2,2))
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = torch.flatten(x, 1, 3)
    x = self.fc1(x)
    return x

net = Net()

#Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)


#Train the network now for 10 epoch
for epoch in range(10):

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 200 == 199:    # print every 200 mini-batches
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
      running_loss = 0.0

print('Finished Training')

#Save the model
torch.save(net.state_dict(), './neural_net.pth')

[1,   200] loss: 0.230
[1,   400] loss: 0.230
[1,   600] loss: 0.229
[1,   800] loss: 0.229
[1,  1000] loss: 0.228
[1,  1200] loss: 0.228
[1,  1400] loss: 0.226
[1,  1600] loss: 0.224
[1,  1800] loss: 0.220
[2,   200] loss: 0.201
[2,   400] loss: 0.155
[2,   600] loss: 0.091
[2,   800] loss: 0.061
[2,  1000] loss: 0.053
[2,  1200] loss: 0.049
[2,  1400] loss: 0.044
[2,  1600] loss: 0.041
[2,  1800] loss: 0.041
[3,   200] loss: 0.040
[3,   400] loss: 0.038
[3,   600] loss: 0.038
[3,   800] loss: 0.037
[3,  1000] loss: 0.037
[3,  1200] loss: 0.037
[3,  1400] loss: 0.036
[3,  1600] loss: 0.034
[3,  1800] loss: 0.035
[4,   200] loss: 0.033
[4,   400] loss: 0.033
[4,   600] loss: 0.032
[4,   800] loss: 0.033
[4,  1000] loss: 0.033
[4,  1200] loss: 0.032
[4,  1400] loss: 0.030
[4,  1600] loss: 0.030
[4,  1800] loss: 0.031
[5,   200] loss: 0.027
[5,   400] loss: 0.029
[5,   600] loss: 0.028
[5,   800] loss: 0.029
[5,  1000] loss: 0.025
[5,  1200] loss: 0.027
[5,  1400] loss: 0.026
[5,  1600] 

KeyboardInterrupt: ignored

In [ ]:
#Lets see how well we did
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    labels = labels.to(device)
    images = images.to(device)

    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct/total))


Accuracy of the network on the test images: 96 %
